In [6]:
pip install transformers

In [7]:
!pip install -U torchtext==0.8.0

In [8]:
pip install emoji

In [92]:
# Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import re, string
import emoji
import nltk
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

from sklearn.metrics import f1_score , confusion_matrix , classification_report
import random


* Pandas digunakan untuk mengubaha dataset ke dataframe manipulasi data dalam bentuk dataframe.
* Numpy digunakan untuk manipulasi data.
*	LabelEncoder digunakan untuk mengubah labels menjadi interger 1 dan 0.
*	NLTK merupakan library yang digunakan untuk melakukan preprocessing dataset.
* Re merupakan library untuk membersihkan dataset
* String merupakan library untuk membersihkan dataset
* Torch meruapak framework untuk tensor
* Train test split untuk membagi train menjadi train dan validation
* tqdm for progress bar
* BertTokenizer untuk melakukan tokenizitaion setelah dilakukan embedding
* BertForSequenceClassification untuk model klasifikasi dari Bert
* F1 Score sebagai evaluation Metric
* Random untuk mendapatkan angka random



#1. Preprocessing

In [10]:
#Import Dataset
train = pd.read_csv('Corona_NLP_train.csv' ,encoding='latin-1')
test = pd.read_csv('Corona_NLP_test.csv')

In [11]:
train = train[['OriginalTweet','Sentiment']]
test = test[['OriginalTweet','Sentiment']]

kita hanya menggunakan column original tweet dan sentiment, maka saya mengdeclare ke train baru

In [12]:
train.drop_duplicates(inplace= True)
test.drop_duplicates(inplace=True)

Kita akan memerika apakah ada data yang duplikat agar menghasilkan klasifikasi yang lebih bagus.

In [13]:
text = train['OriginalTweet']

In [14]:


def clean(text):

    #Change to lower case
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower()

    #Remove Emoji
    re.sub(emoji.get_emoji_regexp(), r"", text) #remove emoji

    #remove urls
    text = re.sub(r'http\S+', " ", text)

    #remove mentions
    text = re.sub(r'@\w+',' ',text)

    #remove hastags
    text = re.sub(r'#\w+', ' ', text)

    #remove digits
    text = re.sub(r'\d+', ' ', text)

    #remove html tags
    text = re.sub('r<.*?>',' ', text)
    
    #remove utf8/ascii
    text = re.sub(r'[^\x00-\x7f]',r'', text)

    #remove banned words
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
   
    return text

In [15]:
train['Cleaned'] = train['OriginalTweet'].apply(lambda x: clean(x))
test['Cleaned'] = test['OriginalTweet'].apply(lambda x: clean(x))

Pada bagian ini , kita akan membersihkat dataset. Dataset ini diambil dari tweet oleh karena itu banyak bagian dalam tweet tesebut yang akan kitab uang seperti emoji, url, mention, hastags, digits, html tags, dan huruf yang tidak biasa. Pertama-tama kita akan memasukan originaltweet kedalam text lalu kita akan memasukan text kedalam function yang kita buat. Kita akan menghapus bagian bagian yang tidak diinginkan dengan library re. setelah itu kita akan menaruh text yang sudah dibersihkan ke dalam column baru.

In [16]:
text_len_test = []
for text in test.Cleaned:
    tweet_len = len(text.split())
    text_len_test.append(tweet_len)

test['text_len'] = text_len_test


In [17]:
text_len = []
for text in train.Cleaned:
    tweet_len = len(text.split())
    text_len.append(tweet_len)

train['text_len'] = text_len

In [18]:
train = train[train['text_len'] > 4]
test = test[test['text_len'] > 4]

Pada bagian ini kita akan menghapus bagian yang memiliki Panjang text kurang dari 4, kita akan menghitung Panjang masing masing tweet lalu memasukan panjangnya kedalam sebuah kolom text_len.

In [19]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [20]:
token_lens = []

for txt in train['Cleaned'].values:
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
    
max_len=np.max(token_lens)

In [21]:
train['token_lens'] = token_lens
train = train.sort_values(by='token_lens', ascending=False)
train.head(20)

,OriginalTweet,Sentiment,Cleaned,text_len,token_lens
37156,11 Nisan Cumartesi itibariyle ?BBÂnin tÃ¼m hi...,Neutral,nisan cumartesi itibariyle bbnin tm hizmetle...,37,96
1638,ZÃ¡sah policie proti spekulant?m s rouÂkami. ...,Neutral,zsah policie proti spekulantm s roukami na mj ...,37,91
16632,Bir mÃ¼ddettir sÃ¼permarketlerin lojistik hizm...,Neutral,bir mddettir spermarketlerin lojistik hizmeti ...,33,89
14593,"Na, wer war denn da am Werk? Gestern Nachmitta...",Extremely Negative,na wer war denn da am werk gestern nachmittag ...,37,80
27005,Supermercados Econo confirman que un empleado ...,Neutral,supermercados econo confirman que un empleado ...,37,76
11213,Keine WertgegenstÃ¤nde im Fahrzeug lassen! - D...,Negative,keine wertgegenstnde im fahrzeug lassen diese...,31,75
4844,Impct of #coronavirus i hve sen hw civilizd pp...,Extremely Negative,impct of i hve sen hw civilizd ppl fighting ...,45,75
10514,the new play Cannabis sales and prices are sky...,Positive,the new play cannabis sales and prices are sky...,41,73
14755,?????????? |\r\r\n\r\r\nCiudadano brasileÃ±o ?...,Neutral,ciudadano brasileo muestra su sorpresa pos...,36,72
9027,La escena se repite alrededor del mundo: desde...,Neutral,la escena se repite alrededor del mundo desde ...,37,70


Pada bagian ini kita akan menghapus tweet yang memiliki bahasa lain dengan memerikasa 

In [22]:
train = train.iloc[12:]
train.head()

,OriginalTweet,Sentiment,Cleaned,text_len,token_lens
29090,i want to go shopping i want to walk w/o the t...,Extremely Positive,i want to go shopping i want to walk wo the th...,62,69
26976,If you read someone being bullish on these pri...,Neutral,if you read someone being bullish on these pri...,47,69
40342,likeit Gallinago Gel cocaine has hiked in pric...,Neutral,likeit gallinago gel cocaine has hiked in pric...,53,69
2825,"They say that"" The Fijian Competition and Cons...",Extremely Negative,they say that the fijian competition and consu...,46,69
722,I'm just saying I work at a Shoppers Drug Mart...,Extremely Negative,im just saying i work at a shoppers drug mart ...,61,69


In [23]:
token_lens_test = []

for txt in test['Cleaned'].values:
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens_test.append(len(tokens))
    
max_len=np.max(token_lens_test)

In [24]:
test['token_lens'] = token_lens_test
test = test.sort_values(by='token_lens', ascending=False)
test.head(10) 

,OriginalTweet,Sentiment,Cleaned,text_len,token_lens
2383,Ahora seguid llorando por el papel higiénico (...,Negative,ahora seguid llorando por el papel higinico qu...,44,94
345,Informoval jsem zástupce vech obchodních ?et?...,Neutral,informoval jsem zstupce vech obchodnch etzc o ...,30,85
1485,DTF-Dont Touch Face\r\r\nDWBH-Do Wash Both Ha...,Extremely Negative,dtfdont touch face dwbhdo wash both hands gtfo...,41,74
3505,Stop misusing ur privilege amp grow up Some1 c...,Positive,stop misusing ur privilege amp grow up some c...,57,71
1209,Im in the DC/Maryland/Virginia (DMV) area &am...,Positive,im in the dcmarylandvirginia dmv area amphave ...,43,71
2740,COVID-19: Your government will save ITSELF not...,Positive,covid your government will save itself not yo...,43,69
855,"Lidl is total chaos, queues as long as the ais...",Extremely Negative,lidl is total chaos queues as long as the aisl...,62,69
1789,"For those that are cashlong, patient,calm&amp;...",Extremely Positive,for those that are cashlong patientcalmamphave...,43,68
2918,obyrne EVAUK actionhunger may food union spark...,Extremely Negative,obyrne evauk actionhunger may food union spark...,55,66
1343,I usually stock up on sanitizer like it's doom...,Negative,i usually stock up on sanitizer like its dooms...,49,65


In [25]:
test = test.iloc[2:]

Pada Bagian ini kita akan melakukan hal yang sama pada bagian test kita.

In [26]:
#LabelEncoder
le = LabelEncoder()
train.Sentiment = le.fit_transform(train.Sentiment)
test.Sentiment = le.fit_transform(test.Sentiment)


Pada bagian ini kita akan mengubah sentiment menjadi angka agar dapat dimenegerti oleh mesing. Kita akan memakai label encoder. 
* 0 = Extremely Positive
* 1 = Extremely Negative
* 2 = Negative
* 3 = Neutral
* 4 = Positive


In [27]:
train = train.drop(columns=['OriginalTweet', 'text_len','token_lens'])
test = test.drop(columns=['OriginalTweet', 'text_len','token_lens'])

Pada bagian ini kita akan menghapus bagian yang tidak dipakai yaitu original tweet text len dan token lens




In [28]:


X_train, X_val, y_train, y_val = train_test_split(train.index.values, 
                                                  train.Sentiment.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=train.Sentiment.values)

train['data_type'] = ['not_set']*train.shape[0]

train.loc[X_train, 'data_type'] = 'train'
train.loc[X_val, 'data_type'] = 'val'

train.groupby(['Cleaned', 'Sentiment', 'data_type']).count()

,,
Cleaned,Sentiment,data_type
be save follow me and my other account call,1,train
be save follow me and my other account inst,1,val
economics professors why you should let prices rise during a buying panic,2,train
tag a mate to raise morale,3,train
our nations new logo i think that says it all,3,train
...,...,...
zoom stunts a toilet paper relay,3,train
zoom takes lead over microsoft teams as coronavirus keeps americans at home,3,train
zoomers to boomers teenagers create online shopping service for isolated seniors,2,train


Pada bagian ini, kita akan melakukan train test split untuk mendapatkan train and validation. Kita akan membagi rata ratio kelas train dan validation agar mendapatkan hasil yang bagus untuk semua kelas.

In [29]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [83]:
encoded_data_train = tokenizer.batch_encode_plus(
    train[train.data_type=='train'].Cleaned.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    train[train.data_type=='val'].Cleaned.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    test.Cleaned.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train[train.data_type=='train'].Sentiment.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(train[train.data_type=='val'].Sentiment.values)



/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [84]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Pada bagian ini kita akan melakukan tokenisasi dengan bert. Tokenisasi adalah proses untuk mengambil teks mentah dan dipecah menjadi token, yang merupakan data numerik untuk mewakili kata-kata. kami menggunakan fungsi yang disebut batch_encode_plus , dan kami akan melanjutkan pelatihan dan validasi data secara terpisah. Parameter pertama di dalam fungsi di atas adalah teks tweet yang sudah dibersihkan. add_special_tokens=True berarti urutan akan dikodekan dengan token khusus relatif terhadap modelnya.Saat mengelompokkan urutan bersama-sama, kami menetapkan return_attention_mask=True, sehingga akan mengembalikan topeng perhatian sesuai dengan tokenizer spesifik yang ditentukan oleh atribut max_length. Kami juga ingin memasukkan semua text ke panjang maksimum tertentu. return_tensors='pt' untuk mengembalikan PyTorch.
Dan kemudian kita perlu membagi data menjadi input_ids, attention_masks dan labels. Akhirnya, setelah kami mendapatkan kumpulan data yang dikodekan, kami dapat membuat data pelatihan dan data validasi.

#3. Build Model

In [32]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=5,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Disini kita akan membuat model pre trained bert kita yang menggunakan bert for sequence classification. jadi satu urutan akan diklasifikasikan ke salah satu dari lima label (yaitu konferensi). bert-base-uncased adalah model pra-pelatihan yang lebih kecil. Menggunakan num_labels untuk menunjukkan jumlah label output. Kami tidak terlalu peduli dengan output_attentions. Kami juga tidak membutuhkan output_hidden_states.

In [86]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)



In [34]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [35]:
epochs = 3

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [64]:


def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

    confusion_matrix(y_true, y_preds)


Untuk mengukur keberhasilan model kita menggunakan performance metrics f1_score dan accuracy per kelas.

In [37]:

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [39]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [40]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/11488 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.8968424973738421
Validation loss: 0.6879039460867074
F1 Score (Weighted): 0.8207793406783179


Epoch 2:   0%|          | 0/11488 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.6291481450731925
Validation loss: 0.6741038340828287
F1 Score (Weighted): 0.8538665345541393


Epoch 3:   0%|          | 0/11488 [00:00<?, ?it/s]


Epoch 3
Training loss: 0.4379016652603575
Validation loss: 0.6916889765969647
F1 Score (Weighted): 0.8687730872739046


Pada bagian ini kita akan mengtraining model. Kita akan memakai GPU karenan memakan waktu yang cukup lama. Kita juga mebuat fungsi evalution untuk melihat performa dari train dan validating model.

In [41]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=5,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [78]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_3.model', map_location=torch.device('cpu')))

<All keys matched successfully>

#4. Evaluation

In [88]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [89]:
label_dict = {
    "Extremely Positive": 0,
    "Extremely Negative" : 1,
    "Negative" : 2,
    "Neutral" : 3,
    "Positive" : 4
}
label_dict

{'Extremely Negative': 1,
 'Extremely Positive': 0,
 'Negative': 2,
 'Neutral': 3,
 'Positive': 4}

In [90]:
accuracy_per_class(predictions, true_vals)

Class: Extremely Positive
Accuracy: 740/820

Class: Extremely Negative
Accuracy: 911/992

Class: Negative
Accuracy: 1265/1477

Class: Neutral
Accuracy: 938/1099

Class: Positive
Accuracy: 1430/1694



Bisa dilihat dari accuracy per kelas bahwa model paling bisa mengklasifikasikan extremely negative.

In [91]:
preds_flat = np.argmax(predictions, axis=1).flatten()
confusion_matrix(true_vals,preds_flat)  

array([[ 740,    0,   77,    2,    1],
       [   1,  911,    4,    0,   76],
       [ 125,    2, 1265,   35,   50],
       [   0,    2,   77,  938,   82],
       [   1,  131,   88,   44, 1430]])

Berikut merupakan confusion matrix dari model saya. Terlihat bahwa beberapa kelas masih terbalik terutama extremely positive dengan negative serta extremely negative dengan positive.

In [94]:
print(classification_report(true_vals, preds_flat))

              precision    recall  f1-score   support

           0       0.85      0.90      0.88       820
           1       0.87      0.92      0.89       992
           2       0.84      0.86      0.85      1477
           3       0.92      0.85      0.89      1099
           4       0.87      0.84      0.86      1694

    accuracy                           0.87      6082
   macro avg       0.87      0.87      0.87      6082
weighted avg       0.87      0.87      0.87      6082



Bisa dilihat dari classification report diatas , akurasi model adalah 0.87. Model sudah dapat mengklasifikasikan dengan baik.